In [1]:
import pandas as pd

In [2]:
intake24_df = pd.read_csv('files/intake24_survey_file.csv')

In [3]:
print(intake24_df)

                                 Survey ID  User ID                Start time  \
0     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
1     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
2     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
3     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
4     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
...                                    ...      ...                       ...   
5749  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5750  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5751  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5752  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5753  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   

               Submission t